# Week 12: Advanced Backtesting & Deployment - Theory

## 🎯 What You'll Learn This Week (The Final Boss!)

Congratulations! You've made it to the final week. This is where theory meets reality.

**The Hard Truth:** Most backtested strategies fail in live trading. Why?
- They didn't account for transaction costs
- They overfitted to historical data
- They had unrealistic assumptions

**By the end of this week, you'll know how to:**
- Test strategies like a real quant (walk-forward, not random splits!)
- Account for ALL the costs that eat your profits
- Size positions to survive drawdowns
- Deploy models to production

**Why This Matters:**
> "A strategy that makes 10% in backtest but loses money live is worthless. A strategy that makes 2% in backtest AND 2% live is gold." - Quantitative PM

---

## Part 1: Walk-Forward Optimization

### 🤔 Why Walk-Forward? (The Real Reason)

**The Problem with Normal Backtesting:**
Imagine you're in 2020 and you test a strategy on 2010-2020 data. Your model "knows" about COVID, the 2018 vol spike, etc. But in 2010, those hadn't happened yet!

**Walk-forward simulates ACTUAL trading:**
- Train on 2010-2015, test on 2016
- Train on 2010-2016, test on 2017
- Train on 2010-2017, test on 2018... and so on

This way, you only test on data the model has NEVER seen.

**Visual Representation:**
```
Time -->   [======Train======][Test]
              [======Train======][Test]
                 [======Train======][Test]
```

### Key Parameters (In Plain English):
- **Training window**: How much history to learn from (e.g., 3 years)
- **Test window**: How long to test before retraining (e.g., 3 months)
- **Step size**: How often to retrain (e.g., monthly)

### Expanding vs Rolling Window - Which to Use?

| Method | What It Does | When to Use It |
|--------|-------------|----------------|
| **Expanding** | Keeps adding data | Markets with lasting patterns |
| **Rolling** | Fixed window, drops old data | Markets that change fast |

**Pro Tip:** Most quants use expanding windows for general strategies, rolling for high-frequency/regime-dependent models.

---

## Part 2: Transaction Costs (The Profit Killer!)

### ⚠️ Why This Section is CRITICAL

Many beginners build strategies showing 20% annual returns... then lose money in live trading. Why? **TRANSACTION COSTS.**

Every time you trade, you pay a "tax." If you trade too often, these taxes eat all your profits.

### Types of Trading Costs (Know Your Enemy):

**1. Commission/Fees** 💰
> The obvious cost - what your broker charges
- Typical: $0.005/share or 0.01-0.1% per trade
- Interactive Brokers: ~$1-5 per trade

**2. Bid-Ask Spread** 📊
> The hidden cost most beginners forget!
- Apple (liquid): $150.00 bid / $150.01 ask = 0.007% cost
- Small cap (illiquid): $10.00 bid / $10.10 ask = 1% cost!!

**3. Market Impact** 📈
> YOUR trade moves the price against you
- Small trade: negligible
- Large trade ($1M+): you're buying so much, price rises BEFORE you finish

**4. Slippage** ⏱️
> Price changes while your order executes
- You want $100, you get $100.05

### The Cost Formula:
$$\text{Total Cost} = \text{Commission} + \frac{\text{Spread}}{2} + \text{Impact} + \text{Slippage}$$

### Real Example:
| Cost Type | Amount |
|-----------|--------|
| Commission | 0.01% |
| Half-spread | 0.025% |
| Slippage | 0.01% |
| **Total per trade** | **0.045%** |
| **Round trip (buy + sell)** | **0.09%** |

If you trade daily, that's 0.09% × 252 = **22.7% per year** eaten by costs!

### Turnover Calculation:
$$\text{Turnover} = \frac{\sum |w_t - w_{t-1}|}{2}$$

Where $w_t$ is the portfolio weight at time $t$.

---

## Part 3: Position Sizing

### Kelly Criterion:
Optimal fraction of capital to bet:

$$f^* = \frac{p \cdot b - q}{b} = \frac{p(b+1) - 1}{b}$$

Where:
- $p$ = win probability
- $q$ = 1 - p (loss probability)
- $b$ = odds (win/loss ratio)

### Half-Kelly:
In practice, use **half Kelly** ($f^*/2$) to reduce volatility and account for estimation error.

### Volatility Targeting:
Scale positions to target volatility:

$$\text{Position Size} = \frac{\sigma_{target}}{\sigma_{asset}}$$

### Risk Budgeting:
- Equal risk contribution
- Risk parity across assets
- Maximum position limits

---

## Part 4: Realistic Backtesting

### Common Backtesting Pitfalls:

1. **Look-ahead bias**: Using future data in decisions
2. **Survivorship bias**: Only testing on surviving assets
3. **Data snooping**: Overfitting to historical patterns
4. **Selection bias**: Cherry-picking favorable periods

### Best Practices:

1. **Use point-in-time data**: No future information
2. **Include delisted securities**: Account for failures
3. **Multiple validation periods**: Out-of-sample + out-of-time
4. **Realistic assumptions**: Conservative cost estimates

### Performance Attribution:
Break down returns by:
- Market timing
- Stock selection
- Factor exposure

---

## Part 5: Risk Management

### Key Risk Metrics:

**Value at Risk (VaR)**:
Maximum expected loss at confidence level:
$$\text{VaR}_{\alpha} = -\mu + \sigma \cdot z_{\alpha}$$

**Conditional VaR (CVaR/Expected Shortfall)**:
Average loss beyond VaR:
$$\text{CVaR}_{\alpha} = E[X | X \leq \text{VaR}_{\alpha}]$$

**Maximum Drawdown**:
$$\text{MDD} = \max_{t \in [0,T]} \left( \max_{s \in [0,t]} P_s - P_t \right)$$

### Drawdown Management:
- Stop trading at drawdown threshold
- Scale down position sizes progressively
- Diversification across strategies

---

## Part 6: Model Deployment

### Deployment Architecture:

```
┌─────────────────────────────────────────────────────────────┐
│                      PRODUCTION SYSTEM                      │
├─────────────────────────────────────────────────────────────┤
│  ┌──────────┐    ┌──────────┐    ┌──────────┐             │
│  │   Data   │───>│  Feature │───>│  Model   │───> Signal  │
│  │  Feed    │    │   Eng.   │    │Inference │             │
│  └──────────┘    └──────────┘    └──────────┘             │
│                                         │                  │
│  ┌──────────┐    ┌──────────┐    ┌──────────┐             │
│  │ Logging  │<───│   Risk   │<───│ Position │<────────────│
│  │Monitor   │    │  Check   │    │  Sizer   │             │
│  └──────────┘    └──────────┘    └──────────┘             │
│                                         │                  │
│                                         v                  │
│                                  ┌──────────┐             │
│                                  │ Execution│             │
│                                  │  Engine  │             │
│                                  └──────────┘             │
└─────────────────────────────────────────────────────────────┘
```

### Key Components:

1. **Data Pipeline**: Real-time data ingestion
2. **Feature Store**: Pre-computed features
3. **Model Server**: Inference endpoint
4. **Risk Engine**: Real-time risk monitoring
5. **Execution**: Order management system

---

## Part 7: Model Monitoring

### What to Monitor:

1. **Model Performance**:
   - Prediction accuracy over time
   - Sharpe ratio (rolling)
   - Hit rate decay

2. **Data Drift**:
   - Feature distribution changes
   - Input data quality
   - Missing data patterns

3. **Concept Drift**:
   - Target distribution shift
   - Regime changes
   - Structural breaks

### Retraining Triggers:
- Scheduled (weekly/monthly)
- Performance degradation
- Significant drift detected

---

## Week 12 Daily Notebooks

| Day | Topic | Key Concepts |
|-----|-------|-------------|
| 1 | Walk-Forward Validation | Expanding/rolling windows, anchored vs sliding |
| 2 | Transaction Costs | Commission, spread, slippage, turnover |
| 3 | Position Sizing | Kelly criterion, volatility targeting |
| 4 | Risk Management | VaR, CVaR, drawdown management |
| 5 | Realistic Backtesting | Bias avoidance, validation |
| 6 | Model Deployment | Inference pipeline, monitoring |
| 7 | Complete System | Production-ready trading system |

---

## Summary

### Key Takeaways:

1. **Walk-Forward**: The only realistic validation for time series
2. **Transaction Costs**: Often determine strategy viability
3. **Position Sizing**: Half-Kelly with volatility targeting
4. **Risk Management**: Drawdowns kill strategies
5. **Monitoring**: Models degrade without maintenance

### Professional Checklist:
- [ ] Walk-forward validation implemented
- [ ] Transaction costs modeled
- [ ] Position sizing rules defined
- [ ] Risk limits set
- [ ] Monitoring dashboard ready
- [ ] Retraining pipeline automated

---